In [1]:
import pandas as pd
import numpy as np

In [2]:
!ls


cbow_harry_potter.ipynb  harry_potter.txt	 models       word2vec.h5
CBOW_keras.ipynb	 India_after_gandhi.txt  Reviews.csv


In [3]:
corpus = pd.read_fwf('harry_potter.txt',names = ['Text'])

In [4]:
corpus.Text.shape

(48305,)

In [5]:
corpus.Text[45090]

'"Shut your mouth!" bellowed Ron, bypassing red and turning maroon.'

In [6]:
import re
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    
    # remove special characters and digits if any
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

corpus['Text'] = corpus['Text'].apply(lambda x:pre_process(x))

In [7]:
corpus.Text[10]

'nearly twice the usual amount of neck which came in very useful as she'

In [8]:
import nltk
from keras.preprocessing import text
from keras.utils import np_utils
from keras.preprocessing import sequence
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda


/home/sourav/anaconda3/envs/Tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(corpus.Text)
word2id = tokenizer.word_index

# build vocabulary of unique words
word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in corpus.Text]

vocab_size = len(word2id)
embed_size = 200
window_size = 5

print('Vocabulary Size:', vocab_size)


Vocabulary Size: 16696


In [10]:
cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='adam')

# view model summary
cbow.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 200)           3339200   
_________________________________________________________________
lambda_1 (Lambda)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16696)             3355896   
Total params: 6,695,096
Trainable params: 6,695,096
Non-trainable params: 0
_________________________________________________________________


In [11]:
def generate_context_word_pairs(corpus, window_size, vocab_size):
    context_length = window_size*2 
    for doc in corpus:
        sentence_length = len(doc)
        for index, word in enumerate(doc):
            context_words = []
            label_word   = []            
            start = index - window_size
            end = index + window_size + 1
            
            context_words.append([doc[i] 
                                 for i in range(start, end) 
                                 if 0 <= i < sentence_length 
                                 and i != index])
            label_word.append(word)

            x = sequence.pad_sequences(context_words, maxlen=context_length)
            y = np_utils.to_categorical(label_word, vocab_size)
            yield (x, y)

In [12]:
a = generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size)

In [13]:
from tqdm import tqdm_notebook
# counter = 0
# for each in tqdm_notebook(a):
#     counter += 1

In [14]:
i = 0
for epoch in range(1, 6):
    loss = 0.0
    for x, y in tqdm_notebook(generate_context_word_pairs(
        corpus=wids, window_size=window_size, vocab_size=vocab_size), total=522303):
        i += 1
        loss += cbow.train_on_batch(x, y)
        if i % 10000 == 0:
            print('At iteration {} cumulative loss {}'.format(i, loss))
    # Save the model at each epoch 
    cbow.save('./models/word2vec_hp_{}.h5'.format(str(epoch)))
    print('Epoch:', epoch, '\tTotal Loss:', loss)

At iteration 10000 cumulative loss 72658.8157697916
At iteration 20000 cumulative loss 144904.48578679562
At iteration 30000 cumulative loss 216680.86483860016
At iteration 40000 cumulative loss 291723.59290006757
At iteration 50000 cumulative loss 364483.39373322576
At iteration 60000 cumulative loss 435933.8864511885
At iteration 70000 cumulative loss 507180.83226618916
At iteration 80000 cumulative loss 577609.8925533427
At iteration 90000 cumulative loss 653302.9688486143
At iteration 100000 cumulative loss 731873.7930505725
At iteration 110000 cumulative loss 809244.8884728827
At iteration 120000 cumulative loss 887483.3859989529
At iteration 130000 cumulative loss 962858.6891389012
At iteration 140000 cumulative loss 1036665.7088488787
At iteration 150000 cumulative loss 1110467.8008727632
At iteration 160000 cumulative loss 1182081.934012476
At iteration 170000 cumulative loss 1256739.8789029964
At iteration 180000 cumulative loss 1334610.2783687362
At iteration 190000 cumulativ

At iteration 530000 cumulative loss 56750.061166964006
At iteration 540000 cumulative loss 130504.27476649646
At iteration 550000 cumulative loss 203640.5443869676
At iteration 560000 cumulative loss 276743.1440894155
At iteration 570000 cumulative loss 346505.78682181914
At iteration 580000 cumulative loss 416653.9650032243
At iteration 590000 cumulative loss 485539.15683732135
At iteration 600000 cumulative loss 552090.8111180415
At iteration 610000 cumulative loss 622723.3617439581
At iteration 620000 cumulative loss 696779.6769231142
At iteration 630000 cumulative loss 770830.7308661082
At iteration 640000 cumulative loss 845669.5122698555
At iteration 650000 cumulative loss 916580.5551464028
At iteration 660000 cumulative loss 988036.5084068746
At iteration 670000 cumulative loss 1058498.0451398003
At iteration 680000 cumulative loss 1127426.4048933296
At iteration 690000 cumulative loss 1197621.8050823081
At iteration 700000 cumulative loss 1270781.1695510102
At iteration 710000 

At iteration 1050000 cumulative loss 37908.91196644679
At iteration 1060000 cumulative loss 108821.42273388109
At iteration 1070000 cumulative loss 179530.57302266225
At iteration 1080000 cumulative loss 248445.48249529622
At iteration 1090000 cumulative loss 316751.21700006863
At iteration 1100000 cumulative loss 384529.0644330119
At iteration 1110000 cumulative loss 450107.9173670053
At iteration 1120000 cumulative loss 515770.0750598665
At iteration 1130000 cumulative loss 583724.5303665882


KeyboardInterrupt: 